In [1]:
import os
import csv
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [2]:
if os.path.exists("marvel.csv"):
    os.remove("marvel.csv")
api.dataset_download_file('dannielr/marvel-superheroes', 'charcters_stats.csv')

True